#Bigquery for billing doc
https://cloud.google.com/billing/docs/how-to/bq-examples

How to use label

https://cloud.google.com/resource-manager/docs/creating-managing-labels

# Init BQ Auth

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
project_id='your-project-id' #@param{type:"string"}
%load_ext google.colab.data_table

Authenticated


# GCE Cost Group by SKU and day


In [ ]:
%%bigquery --project {project_id}
SELECT
  EXTRACT (DATE
  FROM
    usage_start_time AT TIME ZONE "Asia/Shanghai") AS Date,
  sku.description AS SKU,
  SUM(usage.amount) AS Amount,
  usage.unit AS Unit,
  SUM(cost) AS Cost
FROM
  ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771
WHERE
  service.description = "Compute Engine" and Cost > 0
GROUP BY
  Date,
  SKU,
  Unit
ORDER BY
  Date ASC, Cost DESC

,Date,SKU,Amount,Unit,Cost
0,2020-07-08,N1 Predefined Instance Core running in Mumbai,1.295995e+05,seconds,1.366915
1,2020-07-08,N1 Predefined Instance Ram running in Mumbai,5.218377e+14,byte-seconds,0.686879
2,2020-07-08,HTTP Load Balancing: Global Forwarding Rule Mi...,3.313081e+04,seconds,0.230076
3,2020-07-08,Storage PD Capacity in Mumbai,3.573958e+14,byte-seconds,0.005964
4,2020-07-08,Network Internet Egress from Mumbai to China,8.924038e+06,bytes,0.001910
...,...,...,...,...,...
116,2020-07-13,Network Internet Egress from Singapore to EMEA,8.571500e+04,bytes,0.000011
117,2020-07-13,Network Internet Egress from Mumbai to APAC,1.060390e+05,bytes,0.000011
118,2020-07-13,Network Internet Egress from Mumbai to China,4.021900e+04,bytes,0.000008
119,2020-07-13,Network Internet Egress from Singapore to Amer...,3.342400e+04,bytes,0.000006


# GCE Cost Group by day, label and SKU


In [ ]:
%%bigquery --project {project_id}
SELECT
  EXTRACT (DATE
  FROM
    usage_start_time AT TIME ZONE "Asia/Shanghai") AS Date,
  label.value AS label_value,
  sku.description as SKU,
  SUM(usage.amount) AS Amount,
  usage.unit AS Unit,
  sum(cost) as Cost
FROM
  `ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771`,
  UNNEST(labels) AS label
WHERE
  service.description = "Compute Engine" 
  and cost > 0 
  and label.key = "gce_name"
GROUP BY
Date, label_value, SKU, Unit
ORDER BY
Date, label_value, SKU


,Date,label_value,SKU,Amount,Unit,Cost
0,2020-07-12,cdn_bs_01,N1 Predefined Instance Core running in Singapore,1.295991e+05,seconds,1.403952
1,2020-07-12,cdn_bs_01,N1 Predefined Instance Ram running in Singapore,5.218383e+14,byte-seconds,0.705510
2,2020-07-12,cdn_bs_01,Network Internet Egress from Singapore to APAC,5.679135e+07,bytes,0.007403
3,2020-07-12,cdn_bs_01,Network Internet Egress from Singapore to Amer...,8.705400e+04,bytes,0.000013
4,2020-07-12,cdn_bs_01,Network Internet Egress from Singapore to China,1.231620e+05,bytes,0.000025
5,2020-07-12,cdn_bs_01,Network Internet Egress from Singapore to EMEA,1.711910e+05,bytes,0.000021
6,2020-07-12,cdn_bs_01,Storage PD Capacity in Singapore,1.043677e+15,byte-seconds,0.015967
7,2020-07-12,cdn_bs_02,N1 Predefined Instance Core running in Singapore,1.295991e+05,seconds,1.403953
8,2020-07-12,cdn_bs_02,N1 Predefined Instance Ram running in Singapore,5.218383e+14,byte-seconds,0.705509
9,2020-07-12,cdn_bs_02,Network Internet Egress from Singapore to APAC,4.610209e+07,bytes,0.006010


# GCE Cost Group by month, label and SKU


In [ ]:
%%bigquery --project {project_id}
SELECT
  invoice.month AS Month,
  label.value AS label_value,
  sku.description as SKU,
  SUM(usage.amount) AS Amount,
  usage.unit AS Unit,
  sum(cost) as Cost
FROM
  `ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771`,
  UNNEST(labels) AS label
WHERE
  service.description = "Compute Engine" 
  and cost > 0 
  and label.key = "gce_name"
GROUP BY
Month, label_value, SKU, Unit
ORDER BY
Month DESC, label_value, SKU


,Month,label_value,SKU,Amount,Unit,Cost
0,202007,cdn_bs_01,N1 Predefined Instance Core running in Singapore,1.270848e+06,seconds,13.767155
1,202007,cdn_bs_01,N1 Predefined Instance Ram running in Singapore,5.117111e+15,byte-seconds,6.918169
2,202007,cdn_bs_01,Network Inter Zone Egress,3.135770e+05,bytes,0.000002
3,202007,cdn_bs_01,Network Internet Egress from Singapore to APAC,4.978532e+08,bytes,0.064901
4,202007,cdn_bs_01,Network Internet Egress from Singapore to Amer...,3.896640e+05,bytes,0.000061
5,202007,cdn_bs_01,Network Internet Egress from Singapore to China,1.171274e+06,bytes,0.000246
6,202007,cdn_bs_01,Network Internet Egress from Singapore to EMEA,1.128000e+06,bytes,0.000144
7,202007,cdn_bs_01,Storage PD Capacity in Singapore,1.023422e+16,byte-seconds,0.156565
8,202007,cdn_bs_02,N1 Predefined Instance Core running in Singapore,1.270848e+06,seconds,13.767154
9,202007,cdn_bs_02,N1 Predefined Instance Ram running in Singapore,5.117111e+15,byte-seconds,6.918168


# GCE Cost Group by month and label


In [ ]:
%%bigquery --project {project_id}
SELECT
  invoice.month AS Month,
  label.value AS label_value,
  sum(cost) as Cost
FROM
  `ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771`,
  UNNEST(labels) AS label
WHERE
  service.description = "Compute Engine" 
  and cost > 0 
  and label.key = "gce_name"
GROUP BY
Month, label_value
ORDER BY
Month DESC, label_value


,Month,label_value,Cost
0,202007,cdn_bs_01,20.907243
1,202007,cdn_bs_02,20.887749
2,202007,game_server_01,4.178295


# CDN Cache Egress Cost Group by day

In [ ]:
%%bigquery --project {project_id}
SELECT
  EXTRACT (DATE
  FROM
    usage_start_time AT TIME ZONE "Asia/Shanghai") AS Date,
  service.description AS Service,
  SUM(usage.amount)/1024/1024/1024 AS Amount_GB,
  SUM(cost) AS Cost
FROM
  ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771
WHERE
  service.description = "Cloud CDN"
  AND sku.description LIKE '%Cache Egress to%'
GROUP BY
  Date,
  Service
ORDER BY
  Date ASC

,Date,Service,Amount_GB,Cost
0,2020-07-09,Cloud CDN,0.003341,0.000327
1,2020-07-10,Cloud CDN,0.002870,0.000258
2,2020-07-11,Cloud CDN,26.715723,2.397514
3,2020-07-12,Cloud CDN,62.030724,5.555316
4,2020-07-13,Cloud CDN,66.657272,5.974369
5,2020-07-14,Cloud CDN,185.766324,16.668621
6,2020-07-15,Cloud CDN,184.867255,16.587606
7,2020-07-16,Cloud CDN,129.816190,11.650422
8,2020-07-17,Cloud CDN,324.008279,29.095560
9,2020-07-18,Cloud CDN,991.373800,89.143447


# CDN Cache Fill Cost Group by day

In [ ]:
%%bigquery --project {project_id}
SELECT
  EXTRACT (DATE
  FROM
    usage_start_time AT TIME ZONE "Asia/Shanghai") AS Date,
  service.description AS Service,
  SUM(usage.amount)/1024/1024/1024 AS Amount_GB,
  SUM(cost) AS Cost
FROM
  ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771
WHERE
  service.description = "Cloud CDN"
  AND sku.description LIKE '%Fill%'
GROUP BY
  Date,
  Service
ORDER BY
  Date ASC

,Date,Service,Amount_GB,Cost
0,2020-07-11,Cloud CDN,0.129797,0.008653
1,2020-07-12,Cloud CDN,0.074215,0.004378
2,2020-07-13,Cloud CDN,0.084865,0.005023
3,2020-07-14,Cloud CDN,0.108913,0.006526
4,2020-07-15,Cloud CDN,0.198790,0.011941
5,2020-07-16,Cloud CDN,0.133311,0.007928
6,2020-07-17,Cloud CDN,0.421550,0.025315
7,2020-07-18,Cloud CDN,0.348008,0.020854
8,2020-07-19,Cloud CDN,0.305607,0.018342
9,2020-07-20,Cloud CDN,0.366297,0.021931


# GCE Cache Fill Cost Group by day

In [ ]:
%%bigquery --project {project_id}
SELECT
  EXTRACT (DATE
  FROM
    usage_start_time AT TIME ZONE "Asia/Shanghai") AS Date,
  service.description AS Service,
  sku.description AS SKU,
  SUM(usage.amount)/1024/1024/1024 AS Amount_GB,
  SUM(cost) AS Cost
FROM
  ds_us.gcp_billing_export_v1_019952_EBF7BD_9AF771
WHERE
  service.description = "Compute Engine"
  AND sku.description LIKE '%Network CDN%'
GROUP BY
  Date,
  Service,
  SKU
ORDER BY
  Date ASC,
  Cost DESC

,Date,Service,SKU,Amount_GB,Cost
0,2020-07-09,Compute Engine,Network CDN Asia Pacific Intra-Region Cache Fill,0.003027,0.000181
1,2020-07-09,Compute Engine,Network CDN Cache Fill from Asia Pacific to China,0.000175,0.000012
2,2020-07-10,Compute Engine,Network CDN Asia Pacific Intra-Region Cache Fill,0.002870,0.000172
3,2020-07-11,Compute Engine,Network CDN Asia Pacific Intra-Region Cache Fill,14.347280,0.860762
4,2020-07-11,Compute Engine,Network CDN Cache Fill from Asia Pacific to Ot...,0.339794,0.027129
...,...,...,...,...,...
79,2020-07-22,Compute Engine,Network CDN Cache Fill from Asia Pacific to Ot...,1.818124,0.145350
80,2020-07-22,Compute Engine,Network CDN Cache Fill from Asia Pacific to Eu...,1.374211,0.109822
81,2020-07-22,Compute Engine,Network CDN Cache Fill from Asia Pacific to So...,0.226013,0.018053
82,2020-07-22,Compute Engine,Network CDN Cache Fill from Asia Pacific to Oc...,0.015729,0.002348
